In [1]:
# Rushi Parikh
# CSE 572
# HW 3

In [2]:
#Task 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os

# load the dataset from zip
def load_dataset(zip_path='kmeans_data.zip'):
    # Extracting the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    # Find the CSV file
    csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError("No CSV file found in the zip archive")
    
    # Load the dataset
    data = pd.read_csv(csv_files[0])
    return data.values  # Return as numpy array

# Distance metrics
def euclidean_distance(x, y):
    """Euclidean distance between two points"""
    return np.sqrt(np.sum((x - y) ** 2))

def cosine_distance(x, y):
    """1 - Cosine similarity between two points"""
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    
    # Avoid division by zero
    if norm_x == 0 or norm_y == 0:
        return 1.0
    
    cosine_sim = dot_product / (norm_x * norm_y)
    return 1 - cosine_sim

def generalized_jaccard_distance(x, y):
    """1 - Generalized Jaccard similarity between two points"""
    sum_min = np.sum(np.minimum(x, y))
    sum_max = np.sum(np.maximum(x, y))
    
    # Avoid division by zero
    if sum_max == 0:
        return 1.0
    
    jaccard_sim = sum_min / sum_max
    return 1 - jaccard_sim

# K-means implementation
def kmeans(X, k=3, max_iters=100, distance_metric='euclidean', random_state=None):
    # Set random seed
    if random_state is not None:
        np.random.seed(random_state)
    
    # Get dimensions
    n_samples, n_features = X.shape
    
    # Initialize centroids randomly
    idx = np.random.choice(n_samples, k, replace=False)
    centroids = X[idx]
    
    # Choose the distance function
    if distance_metric == 'euclidean':
        distance_function = euclidean_distance
    elif distance_metric == 'cosine':
        distance_function = cosine_distance
    elif distance_metric == 'jaccard':
        distance_function = generalized_jaccard_distance
    else:
        raise ValueError("Unknown distance metric")
    
    # Initialize labels
    labels = np.zeros(n_samples)
    
    # Main K-means loop
    for _ in range(max_iters):
        # Assign points to nearest centroid
        new_labels = np.zeros(n_samples)
        for i in range(n_samples):
            # Calculate distance to each centroid
            distances = [distance_function(X[i], centroid) for centroid in centroids]
            # Assign to the closest centroid
            new_labels[i] = np.argmin(distances)
        
        # Check for convergence
        if np.all(labels == new_labels):
            break
        
        labels = new_labels
        
        # Update centroids
        for j in range(k):
            cluster_points = X[labels == j]
            if len(cluster_points) > 0:
                centroids[j] = np.mean(cluster_points, axis=0)
    
    return centroids, labels

# Run K-means with each distance metric
def main():
    # Load data
    print("Loading dataset...")
    X = load_dataset('kmeans_data.zip')
    print(f"Dataset shape: {X.shape}")
    
    # Number of clusters
    k = 10
    
    # Run K-means with different distance metrics
    metrics = ['euclidean', 'cosine', 'jaccard']
    results = {}
    
    for metric in metrics:
        print(f"\nRunning K-means with {metric} distance...")
        centroids, labels = kmeans(X, k=k, distance_metric=metric, random_state=42)
        
        # Count points in each cluster
        unique, counts = np.unique(labels, return_counts=True)
        cluster_counts = dict(zip(unique, counts))
        
        print(f"Cluster distribution: {cluster_counts}")
        results[metric] = {'centroids': centroids, 'labels': labels}
    
    return results

if __name__ == "__main__":
    main()

Loading dataset...
Dataset shape: (9999, 784)

Running K-means with euclidean distance...
Cluster distribution: {0.0: 1492, 1.0: 768, 2.0: 1221, 3.0: 770, 4.0: 1446, 5.0: 884, 6.0: 997, 7.0: 724, 8.0: 903, 9.0: 794}

Running K-means with cosine distance...
Cluster distribution: {0.0: 1066, 1.0: 1082, 2.0: 758, 3.0: 1023, 4.0: 772, 5.0: 1248, 6.0: 1119, 7.0: 945, 8.0: 1005, 9.0: 981}

Running K-means with jaccard distance...
Cluster distribution: {0.0: 1321, 1.0: 899, 2.0: 676, 3.0: 703, 4.0: 912, 5.0: 927, 6.0: 799, 7.0: 940, 8.0: 1719, 9.0: 1103}


In [3]:
# Q1 

import numpy as np
import pandas as pd
import zipfile
import os

# Function to load the dataset from zip
def load_dataset(zip_path='kmeans_data.zip'):
    # Extract the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')
    
    # Load data.csv and label.csv
    data = pd.read_csv('data.csv', header=None).values
    labels = pd.read_csv('label.csv', header=None).values.flatten()
    
    return data, labels


# Distance metrics
def euclidean_distance(x, y):
    """Euclidean distance between two points"""
    return np.sqrt(np.sum((x - y) ** 2))

def cosine_distance(x, y):
    """1 - Cosine similarity between two points"""
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    
    # Avoid division by zero
    if norm_x == 0 or norm_y == 0:
        return 1.0
    
    cosine_sim = dot_product / (norm_x * norm_y)
    return 1 - cosine_sim

def generalized_jaccard_distance(x, y):
    """1 - Generalized Jaccard similarity between two points"""
    sum_min = np.sum(np.minimum(x, y))
    sum_max = np.sum(np.maximum(x, y))
    
    # Avoid division by zero
    if sum_max == 0:
        return 1.0
    
    jaccard_sim = sum_min / sum_max
    return 1 - jaccard_sim

# K-means implementation
def kmeans(X, k=3, max_iters=100, distance_metric='euclidean', random_state=None):
    """
    K-means clustering algorithm implementation
    
    Parameters:
    - X: data points (n_samples, n_features)
    - k: number of clusters
    - max_iters: maximum number of iterations
    - distance_metric: 'euclidean', 'cosine', or 'jaccard'
    - random_state: random seed for reproducibility
    
    Returns:
    - centroids: center of clusters
    - labels: cluster assignments for each point
    - sse: sum of squared errors (inertia)
    """
    # Set random seed
    if random_state is not None:
        np.random.seed(random_state)
    
    # Get dimensions
    n_samples, n_features = X.shape
    
    # Initialize centroids randomly
    idx = np.random.choice(n_samples, k, replace=False)
    centroids = X[idx]
    
    # Choose the distance function
    if distance_metric == 'euclidean':
        distance_function = euclidean_distance
    elif distance_metric == 'cosine':
        distance_function = cosine_distance
    elif distance_metric == 'jaccard':
        distance_function = generalized_jaccard_distance
    else:
        raise ValueError("Unknown distance metric")
    
    # Initialize labels
    labels = np.zeros(n_samples)
    
    # Main K-means loop
    for _ in range(max_iters):
        # Assign points to nearest centroid
        new_labels = np.zeros(n_samples)
        for i in range(n_samples):
            # Calculate distance to each centroid
            distances = [distance_function(X[i], centroid) for centroid in centroids]
            # Assign to the closest centroid
            new_labels[i] = np.argmin(distances)
        
        # Check for convergence
        if np.all(labels == new_labels):
            break
        
        labels = new_labels
        
        # Update centroids
        for j in range(k):
            cluster_points = X[labels == j]
            if len(cluster_points) > 0:
                centroids[j] = np.mean(cluster_points, axis=0)
    
    # Calculate SSE (Sum of Squared Errors)
    sse = 0
    for i in range(n_samples):
        centroid = centroids[int(labels[i])]
        # For SSE calculation, we use squared Euclidean distance regardless of the clustering distance metric
        sse += np.sum((X[i] - centroid) ** 2)
    
    return centroids, labels, sse

# Run K-means with each distance metric
def main():
    # Load data
    print("Loading dataset...")
    X, y = load_dataset('kmeans_data.zip')
    print(f"Dataset shape: {X.shape}")
    
    # Determine K from the number of unique categories in y
    if y is not None:
        k = len(np.unique(y))
        print(f"Number of categories in target variable: {k}")
    else:
        # Default to 3 if no target variable is found
        k = 3
        print(f"No target variable found, using default k={k}")
    
    # Run K-means with different distance metrics
    metrics = ['euclidean', 'cosine', 'jaccard']
    results = {}
    
    for metric in metrics:
        print(f"\nRunning K-means with {metric} distance...")
        centroids, labels, sse = kmeans(X, k=k, distance_metric=metric, random_state=42)
        
        # Count points in each cluster
        unique, counts = np.unique(labels, return_counts=True)
        cluster_counts = dict(zip(unique, counts))
        
        print(f"Cluster distribution: {cluster_counts}")
        print(f"Sum of Squared Errors (SSE): {sse:.2f}")
        
        results[metric] = {'centroids': centroids, 'labels': labels, 'sse': sse}
    
    # Compare SSEs and determine the best method
    sse_values = {metric: results[metric]['sse'] for metric in metrics}
    best_metric = min(sse_values, key=sse_values.get)
    
    print("\n--- SSE Comparison ---")
    for metric, sse in sse_values.items():
        print(f"{metric.capitalize()}-K-means SSE: {sse:.2f}")
    
    print(f"\nBased on SSE, the best method is {best_metric.capitalize()}-K-means with SSE = {sse_values[best_metric]:.2f}")
    
    return results

if __name__ == "__main__":
    main()

Loading dataset...
Dataset shape: (10000, 784)
Number of categories in target variable: 10

Running K-means with euclidean distance...
Cluster distribution: {0.0: 855, 1.0: 715, 2.0: 502, 3.0: 947, 4.0: 1391, 5.0: 804, 6.0: 1741, 7.0: 1354, 8.0: 1038, 9.0: 653}
Sum of Squared Errors (SSE): 25413478163.00

Running K-means with cosine distance...
Cluster distribution: {0.0: 926, 1.0: 635, 2.0: 1812, 3.0: 1067, 4.0: 1382, 5.0: 888, 6.0: 730, 7.0: 801, 8.0: 777, 9.0: 982}
Sum of Squared Errors (SSE): 25493586936.00

Running K-means with jaccard distance...
Cluster distribution: {0.0: 950, 1.0: 690, 2.0: 1084, 3.0: 799, 4.0: 1001, 5.0: 881, 6.0: 1258, 7.0: 668, 8.0: 1397, 9.0: 1272}
Sum of Squared Errors (SSE): 25418563596.00

--- SSE Comparison ---
Euclidean-K-means SSE: 25413478163.00
Cosine-K-means SSE: 25493586936.00
Jaccard-K-means SSE: 25418563596.00

Based on SSE, the best method is Euclidean-K-means with SSE = 25413478163.00


In [4]:
# Q2

from collections import Counter
import numpy as np
import pandas as pd
import zipfile

# Load data and labels
with zipfile.ZipFile('kmeans_data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')
X = pd.read_csv('data.csv', header=None).values
y = pd.read_csv('label.csv', header=None).values.flatten()
k = len(np.unique(y))

# Distance functions
def euclidean_distance(x, y): return np.sqrt(np.sum((x - y) ** 2))
def cosine_distance(x, y):
    dot = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return 1 - dot / (norm_x * norm_y + 1e-10)
def generalized_jaccard_distance(x, y):
    return 1 - np.sum(np.minimum(x, y)) / (np.sum(np.maximum(x, y)) + 1e-10)

# Basic KMeans
def kmeans(X, k=3, max_iters=100, distance_metric='euclidean', random_state=42):
    np.random.seed(random_state)
    centroids = X[np.random.choice(len(X), k, replace=False)]
    if distance_metric == 'euclidean':
        dist_fn = euclidean_distance
    elif distance_metric == 'cosine':
        dist_fn = cosine_distance
    elif distance_metric == 'jaccard':
        dist_fn = generalized_jaccard_distance
    labels = np.zeros(len(X))
    for _ in range(max_iters):
        new_labels = np.array([np.argmin([dist_fn(x, c) for c in centroids]) for x in X])
        if np.all(labels == new_labels): break
        labels = new_labels
        for i in range(k):
            points = X[labels == i]
            if len(points) > 0:
                centroids[i] = points.mean(axis=0)
    return centroids, labels

# Run KMeans and store results
results = {}
for metric in ['euclidean', 'cosine', 'jaccard']:
    _, labels = kmeans(X, k=k, distance_metric=metric)
    results[metric] = {'labels': labels}


In [5]:
def compute_accuracy(true_labels, predicted_clusters):
    cluster_to_label = {}
    for cluster_id in np.unique(predicted_clusters):
        indices = np.where(predicted_clusters == cluster_id)[0]
        majority_label = Counter(true_labels[indices]).most_common(1)[0][0]
        cluster_to_label[cluster_id] = majority_label
    predicted_labels = np.array([cluster_to_label[cluster] for cluster in predicted_clusters])
    accuracy = np.mean(predicted_labels == true_labels)
    return accuracy

print("\n--- Accuracy Comparison ---")
for metric in ['euclidean', 'cosine', 'jaccard']:
    acc = compute_accuracy(y, results[metric]['labels'])
    print(f"{metric.capitalize()}-K-means Accuracy: {acc * 100:.2f}%")



--- Accuracy Comparison ---
Euclidean-K-means Accuracy: 59.03%
Cosine-K-means Accuracy: 63.07%
Jaccard-K-means Accuracy: 60.29%


In [6]:
#Q3

import time

def kmeans(X, k=10, max_iters=100, distance_metric='euclidean', random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    n_samples = len(X)
    centroids = X[np.random.choice(n_samples, k, replace=False)]

    if distance_metric == 'euclidean':
        dist_fn = euclidean_distance
    elif distance_metric == 'cosine':
        dist_fn = cosine_distance
    elif distance_metric == 'jaccard':
        dist_fn = generalized_jaccard_distance

    labels = np.zeros(n_samples)
    prev_sse = float('inf')
    start_time = time.time()

    for iteration in range(1, max_iters + 1):
        # Assignment step
        new_labels = np.array([
            np.argmin([dist_fn(x, c) for c in centroids])
            for x in X
        ])

        # Update step
        new_centroids = np.copy(centroids)
        for j in range(k):
            points = X[new_labels == j]
            if len(points) > 0:
                new_centroids[j] = points.mean(axis=0)

        # Compute SSE
        current_sse = sum(np.sum((X[new_labels == i] - new_centroids[i])**2) for i in range(k))

        # Check stop criteria
        if np.allclose(new_centroids, centroids) or current_sse > prev_sse:
            break

        centroids = new_centroids
        labels = new_labels
        prev_sse = current_sse

    end_time = time.time()
    runtime = end_time - start_time

    return centroids, labels, current_sse, iteration, runtime


In [7]:
print("\n--- Convergence Comparison ---")
for metric in ['euclidean', 'cosine', 'jaccard']:
    _, labels, sse, iterations, runtime = kmeans(X, k=k, distance_metric=metric)
    print(f"{metric.capitalize()}-K-means → Iterations: {iterations}, Time: {runtime:.2f} seconds")



--- Convergence Comparison ---
Euclidean-K-means → Iterations: 28, Time: 23.64 seconds
Cosine-K-means → Iterations: 40, Time: 42.05 seconds
Jaccard-K-means → Iterations: 26, Time: 32.00 seconds


In [8]:
#Q4

def kmeans_custom_stop(X, k=10, distance_metric='euclidean', mode='centroid', max_iters=100, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    n_samples = len(X)
    centroids = X[np.random.choice(n_samples, k, replace=False)]

    if distance_metric == 'euclidean':
        dist_fn = euclidean_distance
    elif distance_metric == 'cosine':
        dist_fn = cosine_distance
    elif distance_metric == 'jaccard':
        dist_fn = generalized_jaccard_distance

    labels = np.zeros(n_samples)
    prev_sse = float('inf')

    for iteration in range(1, max_iters + 1):
        new_labels = np.array([
            np.argmin([dist_fn(x, c) for c in centroids]) for x in X
        ])

        new_centroids = np.copy(centroids)
        for j in range(k):
            cluster_points = X[new_labels == j]
            if len(cluster_points) > 0:
                new_centroids[j] = np.mean(cluster_points, axis=0)

        current_sse = sum(np.sum((X[new_labels == i] - new_centroids[i])**2) for i in range(k))

        # Apply stopping condition
        if mode == 'centroid' and np.allclose(new_centroids, centroids):
            break
        elif mode == 'sse' and current_sse > prev_sse:
            break
        elif mode == 'max' and iteration == max_iters:
            break

        centroids = new_centroids
        labels = new_labels
        prev_sse = current_sse

    return current_sse


In [10]:

modes = ['centroid', 'sse', 'max']
metrics = ['euclidean', 'cosine', 'jaccard']

for mode in modes:
    print(f"\nStopping condition: {mode.upper()}")
    for metric in metrics:
        sse = kmeans_custom_stop(X, k=k, distance_metric=metric, mode=mode)
        print(f"{metric.capitalize()}: {sse:.2f}")



Stopping condition: CENTROID
Euclidean: 25410429952.00
Cosine: 25471013508.00
Jaccard: 25490781886.00

Stopping condition: SSE
Euclidean: 25491814091.00
Cosine: 25425739868.00
Jaccard: 25430549479.00

Stopping condition: MAX
Euclidean: 25389181077.00
Cosine: 25427124389.00
Jaccard: 25609657930.00
